# Quick start to KinFragLib

Introduction on how to load and use the fragment library

- Load python packages and utility functions for fragment library:

In [15]:
from pathlib import Path
from util import read_fragment_library

### Read fragment library

In [17]:
# Path to library folder
PATH_TO_LIB = Path('.') / '..' / 'data' / 'fragment_library'

In [30]:
fragment_library = read_fragment_library(PATH_TO_LIB)
fragment_library.keys()

dict_keys(['AP', 'FP', 'SE', 'GA', 'B1', 'B2', 'X'])

*fragment_library* is a dictionary containing a pandas dataframe for each of the six subpockets (AP, FP, SE, GA, B1, B2) and the X pool.